In [101]:
import torch
import random
import numpy as np
import re

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

PATH="/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_words/Professions/Female"
FASTBPE="/home/vzhekova/fastBPE/fast" # path to the fastBPE tool
FAST_ALIGN="/home/vzhekova/fast_align/build/fast_align" # path to the fast_align tool
TERCOM = "/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_male/Perturbation-basedQE"

In [102]:
# check if we can connect to the GPU with PyTorch
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'

Current device: GeForce GTX 1080 Ti


In [103]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_words/Professions/Female


In [6]:
import csv

# Convert text file to csv
with open('en_pro.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split("	") for line in stripped if line)
    with open('en_pro.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        #writer.writerow(('gender', 'position', 'sentence', 'word'))
        writer.writerows(lines)       

In [7]:
# Split words into male and female professions
with open('en_pro.csv') as csv_file, open('male.txt', 'w') as out_male, open('female.txt', 'w')as out_female:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        gender = row[0]
        profession = row[3]
        if (gender == "male"):
            print(profession, end='\n', file=out_male)
        if (gender == "female"):
            print(profession, end='\n', file=out_female) 

In [104]:
# Build sentences
# 20 unique sentences in total
with open('female.txt', 'r') as fin, open('en_original.txt', 'w') as fout:
    for line in fin:
        token = line.strip()
        print('The ' + token + ' was late for work.', end='\n', file=fout)
        
# Modify gender ambiguous words with male gender
with open('female.txt', 'r') as fin, open('en_disambiguated_male.txt', 'w') as fout:
    for line in fin:
        token = line.strip()
        print('The male ' + token + ' was late for work.', end='\n', file=fout)
        
# Modify gender ambiguous words with female gender
with open('female.txt', 'r') as fin, open('en_disambiguated_female.txt', 'w') as fout:
    for line in fin:
        token = line.strip()
        print('The female ' + token + ' was late for work.', end='\n', file=fout)

# Translation English-German

In [105]:
# Tokenization
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer, MosesDetokenizer
from __future__ import print_function

mpn = MosesPunctNormalizer()
mt_en = MosesTokenizer(lang='en')
md_en = MosesDetokenizer(lang='en')

with open('en_original.txt') as fin, open('tok.en_original.en','w') as fout:
    for line in fin:
        tokens = mt_en.tokenize(mpn.normalize(line), return_str=True)
        print(tokens, end='\n', file=fout) 
        
with open('en_disambiguated_male.txt') as fin, open('tok.en_disambiguated_male.en','w') as fout:
    for line in fin:
        tokens = mt_en.tokenize(mpn.normalize(line), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('en_disambiguated_female.txt') as fin, open('tok.en_disambiguated_female.en','w') as fout:
    for line in fin:
        tokens = mt_en.tokenize(mpn.normalize(line), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished tokenizing.')

Finished tokenizing.


In [106]:
# Dividing text into subword units

!$FASTBPE applybpe bpe.en_original.en tok.en_original.en bpecodes.en
!$FASTBPE applybpe bpe.en_disambiguated_male.en tok.en_disambiguated_male.en bpecodes.en
!$FASTBPE applybpe bpe.en_disambiguated_female.en tok.en_disambiguated_female.en bpecodes.en

print('Finished subword.')

Loading codes from bpecodes.en ...
Read 30000 codes from the codes file.
Loading vocabulary from tok.en_original.en ...
Read 140 words (26 unique) from text file.
Applying BPE to tok.en_original.en ...
Modified 140 words from text file.
Loading codes from bpecodes.en ...
Read 30000 codes from the codes file.
Loading vocabulary from tok.en_disambiguated_male.en ...
Read 160 words (27 unique) from text file.
Applying BPE to tok.en_disambiguated_male.en ...
Modified 160 words from text file.
Loading codes from bpecodes.en ...
Read 30000 codes from the codes file.
Loading vocabulary from tok.en_disambiguated_female.en ...
Read 160 words (27 unique) from text file.
Applying BPE to tok.en_disambiguated_female.en ...
Modified 160 words from text file.
Finished subword.


In [107]:
# Binarize text
!fairseq-preprocess \
    --source-lang en \
    --target-lang de \
    --testpref bpe.en_original \
    --only-source \
    --srcdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.en.txt \
    --tgtdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.de.txt \
    --destdir data-bin_original_en-de \
    --workers 8

!fairseq-preprocess \
    --source-lang en \
    --target-lang de \
    --testpref bpe.en_disambiguated_male \
    --only-source \
    --srcdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.en.txt \
    --tgtdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.de.txt \
    --destdir data-bin_disambiguated_male_en-de \
    --workers 8

!fairseq-preprocess \
    --source-lang en \
    --target-lang de \
    --testpref bpe.en_disambiguated_female \
    --only-source \
    --srcdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.en.txt \
    --tgtdict /export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble/dict.de.txt \
    --destdir data-bin_disambiguated_female_en-de \
    --workers 8

print('Finished preprocessing.')

2023-08-03 16:55:48 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_original_en-de', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcd

In [109]:
MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
NBEST = 10
BEAM = 10

In [110]:
# Generate translations
# Beam search
!fairseq-generate data-bin_original_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 64 \
    --memory-efficient-fp16 \
    --remove-bpe > original_en-de.decode_Beam_10.log

!fairseq-generate data-bin_disambiguated_male_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 64 \
    --memory-efficient-fp16 \
    --remove-bpe > disambiguated_male_en-de.decode_Beam_10.log

!fairseq-generate data-bin_disambiguated_female_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 64 \
    --memory-efficient-fp16 \
    --remove-bpe > disambiguated_female_en-de.decode_Beam_10.log

print('Finished translation.')

2023-08-03 17:06:20 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

# Backtranslation German-English

In [111]:
# 'LC_ALL=C sort -V' sorts the results in natural order 
!grep ^H original_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original.txt
!grep ^H disambiguated_male_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_disambiguated_male.txt
!grep ^H disambiguated_female_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_disambiguated_female.txt

In [112]:
# Dividing tokenized text into subword units

!$FASTBPE applybpe bpe.hyp_original.de hyp_original.txt bpecodes.de
!$FASTBPE applybpe bpe.hyp_disambiguated_male.de hyp_disambiguated_male.txt bpecodes.de
!$FASTBPE applybpe bpe.hyp_disambiguated_female.de hyp_disambiguated_female.txt bpecodes.de


print('Finished subword.')

Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_original.txt ...
Read 1572 words (70 unique) from text file.
Applying BPE to hyp_original.txt ...
Modified 1572 words from text file.
Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_disambiguated_male.txt ...
Read 1724 words (60 unique) from text file.
Applying BPE to hyp_disambiguated_male.txt ...
Modified 1724 words from text file.
Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_disambiguated_female.txt ...
Read 1549 words (54 unique) from text file.
Applying BPE to hyp_disambiguated_female.txt ...
Modified 1549 words from text file.
Finished subword.


In [113]:
!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_original \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_original_de-en \
    --workers 8

!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_disambiguated_male \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_disambiguated_male_de-en \
    --workers 8

!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_disambiguated_female \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_disambiguated_female_de-en \
    --workers 8

print('Finished preprocessing.')

2023-08-03 17:48:20 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_original_de-en', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='de', srcd

In [115]:
MODELS="/export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble"
NBEST = 10
BEAM = 10

In [116]:
# Generate backtranslations
!fairseq-generate data-bin_original_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 64 \
    --memory-efficient-fp16 \
    --remove-bpe > original_de-en.decode_Beam_10_backtranslation.log

!fairseq-generate data-bin_disambiguated_male_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 64 \
    --memory-efficient-fp16 \
    --remove-bpe > disambiguated_male_de-en.decode_Beam_10_backtranslation.log

!fairseq-generate data-bin_disambiguated_female_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 64 \
    --memory-efficient-fp16 \
    --remove-bpe > disambiguated_female_de-en.decode_Beam_10_backtranslation.log

print('Finished translation.')

2023-08-03 17:54:10 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [117]:
# 'LC_ALL=C sort -V' sorts the results in natural order 
!grep ^H original_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_back.txt
!grep ^H disambiguated_male_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_disambiguated_male_back.txt
!grep ^H disambiguated_female_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_disambiguated_female_back.txt

In [118]:
# Detokenize text        
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer, MosesDetokenizer
from __future__ import print_function

md_en = MosesDetokenizer(lang='en')

with open('hyp_original_back.txt', encoding='utf8') as fin, open('original_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('hyp_disambiguated_male_back.txt', encoding='utf8') as fin, open('disambiguated_male_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('hyp_disambiguated_female_back.txt', encoding='utf8') as fin, open('disambiguated_female_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished detokenizing.')

Finished detokenizing.


# Statistics on translations

In [119]:
# List with original source sentences
source = []
with open('en_original.txt', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
# List with disambiguated source sentences male
source_disambiguated_male = []
with open('en_disambiguated_male.txt', 'r') as fin:
    for line in fin:
        source_disambiguated_male.append(line.strip())
        
# List with disambiguated source sentences female
source_disambiguated_female = []
with open('en_disambiguated_female.txt', 'r') as fin:
    for line in fin:
        source_disambiguated_female.append(line.strip())        
    
# List with nbest sentences for every source in original
nbest_original = []
counter = 0
temp = []
with open('hyp_original.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10):
            nbest_original.append(temp)
            counter = 0
            temp = []
            
# List with nbest sentences for every source in disambiguated male        
nbest_disambiguated_male = []
with open('hyp_disambiguated_male.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10):
            nbest_disambiguated_male.append(temp)
            counter = 0
            temp = []
            
# List with nbest sentences for every source in disambiguated female        
nbest_disambiguated_female = []
with open('hyp_disambiguated_female.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10):
            nbest_disambiguated_female.append(temp)
            counter = 0
            temp = []
            
print(len(source))
print(len(nbest_original))
print(len(nbest_disambiguated_male))
print(len(nbest_disambiguated_female))

20
20
20
20


## Count unique sentences

In [120]:
# Count unique sentences in source nbest list for each source sentence
def count_unique_sentences(nbest_sentences):
    unique_sent = []
    for source_nbest in nbest_sentences:
        num_values = len(set(source_nbest))
        #print(num_values)
        unique_sent.append(num_values)

    #print(unique_sent)
    return sum(unique_sent)/len(nbest_sentences) # average

In [121]:
# Value should be 10, because beam search generates 10 unique sentences
print(count_unique_sentences(nbest_original))

print(count_unique_sentences(nbest_disambiguated_male))

print(count_unique_sentences(nbest_disambiguated_female))

10.0
10.0
10.0


## Count unique words

In [122]:
# Count unique words in source nbest list for each source sentence of original
# !!! Method is slow
import spacy

def count_unique_words(nbest_sentences):
    sp = spacy.load('en_core_web_sm')
    stopwords = spacy.lang.en.stop_words.STOP_WORDS
    
    unique_words = []
    normalizer = 0 # should normalize based on total number of words, because disambiguated sentences have more words overall
    counter = 0
    for source_nbest in nbest_sentences:
        words = set()
        for sent in source_nbest:
            tokens = sp(sent)
            normalizer += len(tokens)
            for token in tokens:
                #if token.text not in stopwords:    # checking whether the word is a stop word
                    words.add(token.text)
        num_values = len(words)
        unique_words.append(num_values)

        counter += 1
        #print(counter)

    #print(unique_words)
    print('Normalizer: ' + str(normalizer))
    return sum(unique_words)/len(nbest_sentences) # average

In [123]:
print(count_unique_words(nbest_original))

print(count_unique_words(nbest_disambiguated_male))

print(count_unique_words(nbest_disambiguated_female))

Normalizer: 1572
13.9
Normalizer: 1724
13.6
Normalizer: 1549
12.9


# Statistics on backtranslations

In [124]:
# List with original source sentences
source = []
with open('en_original.txt', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
# List with disambiguated source sentences male
source_disambiguated_male = []
with open('en_disambiguated_male.txt', 'r') as fin:
    for line in fin:
        source_disambiguated_male.append(line.strip())
        
# List with disambiguated source sentences female
source_disambiguated_female = []
with open('en_disambiguated_female.txt', 'r') as fin:
    for line in fin:
        source_disambiguated_female.append(line.strip())        
    
# List with nbest sentences for every source in original 
nbest_original = []
counter = 0
temp = []
with open('original_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_original.append(temp)
            counter = 0
            temp = []
            
# List with nbest sentences for every source in disambiguated male
nbest_disambiguated_male = []
with open('disambiguated_male_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_disambiguated_male.append(temp)
            counter = 0
            temp = []
            
# List with nbest sentences for every source in disambiguated female
nbest_disambiguated_female = []
with open('disambiguated_female_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_disambiguated_female.append(temp)
            counter = 0
            temp = []
            
print(len(nbest_original))
print(len(nbest_disambiguated_male))
print(len(nbest_disambiguated_female))

20
20
20


## Source sentence reoccurrence

In [125]:
# Count how many of the source sentences reoccur in the backtranslation
def count_sentence_reoccurrence(source_sentences, nbest_sentences):
    results = []
    counter = 0
    for sent in source_sentences:
        matches = 0
        for target in nbest_sentences[counter]: 
            if (sent == target):
                matches += 1
        results.append(matches)  
        counter += 1

    return sum(x > 0 for x in results)

In [126]:
print(count_sentence_reoccurrence(source, nbest_original))

print(count_sentence_reoccurrence(source_disambiguated_male, nbest_disambiguated_male))

print(count_sentence_reoccurrence(source_disambiguated_female, nbest_disambiguated_female))

20
18
2


## Ambiguous source words reoccurrence


In [127]:
# Extract ambiguous words from source sentences
ambiguous_words = [] 
with open('en_original.txt', 'r') as fin:
    for line in fin:
        tokens = line.split(" ")
        ambiguous_words.append(tokens[1].replace('\n', ''))
        
print(ambiguous_words)
print(len(ambiguous_words))

['accountant', 'assistant', 'attendant', 'auditor', 'baker', 'cashier', 'cleaner', 'clerk', 'counselor', 'designer', 'editor', 'hairdresser', 'housekeeper', 'librarian', 'nurse', 'receptionist', 'secretary', 'tailor', 'teacher', 'writer']
20


In [128]:
# Count how many of the ambiguous words reoccur in the backtranslation
def count_words_reoccurrence(ambiguous_words, nbest_sentences):
    results = []
    counter = 0
    for word in ambiguous_words:
        matches = 0
        for target in nbest_sentences[counter]: 
            if (word in target.split(" ")):
                matches += 1
        results.append(matches)  
        counter += 1

    return sum(x > 0 for x in results)

In [129]:
print(count_words_reoccurrence(ambiguous_words, nbest_original))

print(count_words_reoccurrence(ambiguous_words, nbest_disambiguated_male))

print(count_words_reoccurrence(ambiguous_words, nbest_disambiguated_female))

20
20
20


## Count unique sentences

In [130]:
print(count_unique_sentences(nbest_original))

print(count_unique_sentences(nbest_disambiguated_male))

print(count_unique_sentences(nbest_disambiguated_female))

45.45
51.6
52.05


## Count unique words

In [131]:
print(count_unique_words(nbest_original))

print(count_unique_words(nbest_disambiguated_male))

print(count_unique_words(nbest_disambiguated_female))

Normalizer: 14643
27.85
Normalizer: 16187
27.45
Normalizer: 14502
30.25


# Word alignement (source-translation)

- Input to fast_align must be tokenized and aligned into parallel sentences. 
- Line is a source language sentence and its target language translation, separated by a triple pipe symbol with leading and trailing white space (|||)

In [132]:
def build_alignment_input(sentencesN, sourceIn, targetIn, output):
    # List with original source sentences
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    # List with nbest sentences for every source in original 
    target = []
    counter = 0
    temp = []
    with open(targetIn, 'r') as fin:
        for line in fin:
            temp.append(line.strip())
            counter += 1
            if (counter == 10):
                target.append(temp)
                counter = 0
                temp = []

    #print(len(source))
    #print(len(target))           

    count = 0
    with open(output, 'w') as fout:
        while count < sentencesN:
            for hyp in target[count]:
                print(source[count] + ' ||| ' + hyp, end='\n', file=fout)
            count += 1

In [133]:
build_alignment_input(20, 'tok.en_original.en', 'hyp_original.txt', 'original_source-target_en-de.txt')
build_alignment_input(20, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de.txt')
build_alignment_input(20, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de.txt')

## fast_align

In [134]:
!$FAST_ALIGN -i original_source-target_en-de.txt -d -o -v > original_source-target_en-de_fast-aligned.txt
!$FAST_ALIGN -i disambiguated_male_source-target_en-de.txt -d -o -v > disambiguated_male_source-target_en-de_fast-aligned.txt
!$FAST_ALIGN -i disambiguated_female_source-target_en-de.txt -d -o -v > disambiguated_female_source-target_en-de_fast-aligned.txt

print("Finished alignment.")

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
expected target length = source length * 1.12286
ITERATION 1
  log_e likelihood: -32577
  log_2 likelihood: -46998.6
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.194309
       size counts: 4
ITERATION 2
  log_e likelihood: -3595.04
  log_2 likelihood: -5186.55
     cross entropy: 3.29933
        perplexity: 9.8446
      posterior p0: 0.0472473
 posterior al-feat: -0.153567
       size counts: 4
  1  model al-feat: -0.148315 (tension=4)
  2  model al-feat: -0.150677 (tension=3.89496)
  3  model al-feat: -0.151998 (tension=3.83716)
  4  model al-feat: -0.152721 (tension=3.80577)
  5  model al-feat: -0.153113 (tension=3.78885)
  6  model al-feat: -0.153324 (tension=3.77976)
  7  model al-feat: -0.153437 (tension=3.7749)
  8  model al-feat: -0.153497 (tension=3.77229)
     final tension: 3.7709
ITERATION 3
  log_e likelihood: -3060.31
  log_2 likelihood: -4415.09
     cross entropy: 2.80858
     

In [135]:
import re

# Count unique translated words to the ambiguous words in translations per source sentence
def count_unique_words_alignment_translations(position, sentencesN, sourceIn, translationsIn, alignmentsIn, output):
    
    # Get positions of ambigous words
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
        
    # List with translations
    translations = []
    with open(translationsIn, 'r') as fin:
        for line in fin:
            translations.append(line.strip())
            
            
    
    # Extract alginments of ambiguous words
    lineNumber = 0
    counter = 0
    indices = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                indices.append([int(index) for index in re.findall(regex, line)])
            else:
                indices.append([999])
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = set() # set forces uniqueness
    for translation in translations:
        tokens = translation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                #print(lineNumber)
                #print(tokens[ind])
                #print(ind)
                translated_ambiguous_words.add(tokens[ind])
        lineNumber += 1
        if (lineNumber % 10 == 0):
                translations_ambiguous_words.append(translated_ambiguous_words)
                translated_ambiguous_words = set()

    #print(translations_ambiguous_words)
    #print(len(translations_ambiguous_words))
    
    # Add results to file
    ambiguous_words = []
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0                
    with open(output, 'w') as fout:
        while count < sentencesN:
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(translations_ambiguous_words[count]), end='\n', file=fout)
            count += 1

    unique_translations = 0
    for set_words in translations_ambiguous_words:
        
        ############################################################
        # remove gender info; removing "in" and "e" endings in words
        set_words_new = set()
        for word in set_words:
            word_new = re.sub("in$|e$", "", word)
            #print(word_new)
            set_words_new.add(word_new)
        #print(set_words_new)
        ############################################################
        
        unique_translations += len(set_words_new)
        
    #print(unique_translations)
    return unique_translations/sentencesN # average

In [136]:
print(count_unique_words_alignment_translations(1, 20, 'tok.en_original.en', 'hyp_original.txt', 'original_source-target_en-de_fast-aligned.txt', 'unique-words_translations_original.txt'))
print('======')
print(count_unique_words_alignment_translations(2, 20, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_fast-aligned.txt', 'unique-words_translations_disambiguated_male.txt')) # positions is 2 because of gender word
print('======')
print(count_unique_words_alignment_translations(2, 20, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_fast-aligned.txt', 'unique-words_translations_disambiguated_female.txt'))

2.1
2.0
2.0


## awesome_align

In [137]:
# ??? How to set model correctly
# MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
!awesome-align \
    --output_file "original_source-target_en-de_awesome-aligned.txt" \
    --data_file "original_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "disambiguated_male_source-target_en-de_awesome-aligned.txt" \
    --data_file "disambiguated_male_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "disambiguated_female_source-target_en-de_awesome-aligned.txt" \
    --data_file "disambiguated_female_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

print("Finished alignment.")

Loading the dataset...
Extracting: 200it [00:01, 184.10it/s]
Loading the dataset...
Extracting: 200it [00:00, 206.58it/s]
Loading the dataset...
Extracting: 200it [00:01, 183.21it/s]
Finished alignment.


In [138]:
print(count_unique_words_alignment_translations(1, 20, 'tok.en_original.en', 'hyp_original.txt', 'original_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_original.txt'))
print('======')
print(count_unique_words_alignment_translations(2, 20, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_disambiguated_male.txt')) # positions is 2 because of gender word
print('======')
print(count_unique_words_alignment_translations(2, 20, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_disambiguated_female.txt'))

2.05
1.95
1.95


# Word alignement (translation-backtranslation)

## fast_align

- Input to fast_align must be tokenized and aligned into parallel sentences. 
- Line is a source language sentence and its target language translation, separated by a triple pipe symbol with leading and trailing white space (|||)

In [139]:
build_alignment_input(200, 'hyp_original.txt', 'hyp_original_back.txt', 'original_translation-back_en-de.txt')
build_alignment_input(200, 'hyp_disambiguated_male.txt', 'hyp_disambiguated_male_back.txt', 'disambiguated_male_translation-back_en-de.txt')
build_alignment_input(200, 'hyp_disambiguated_female.txt', 'hyp_disambiguated_female_back.txt', 'disambiguated_female_translation-back_en-de.txt')

- Word alignement

In [140]:
!$FAST_ALIGN -i original_translation-back_en-de.txt -d -o -v > original_translation-back_en-de_fast-aligned.txt
!$FAST_ALIGN -i disambiguated_male_translation-back_en-de.txt -d -o -v > disambiguated_male_translation-back_en-de_fast-aligned.txt
!$FAST_ALIGN -i disambiguated_female_translation-back_en-de.txt -d -o -v > disambiguated_female_translation-back_en-de_fast-aligned.txt

print("Finished alignment.")

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
..
expected target length = source length * 0.935688
ITERATION 1
..
  log_e likelihood: -303451
  log_2 likelihood: -437787
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.195414
       size counts: 20
ITERATION 2
..
  log_e likelihood: -33367.1
  log_2 likelihood: -48138.6
     cross entropy: 3.28748
        perplexity: 9.76406
      posterior p0: 0.0470718
 posterior al-feat: -0.152256
       size counts: 20
  1  model al-feat: -0.17923 (tension=4)
  2  model al-feat: -0.16477 (tension=4.53949)
  3  model al-feat: -0.158637 (tension=4.78977)
  4  model al-feat: -0.155641 (tension=4.9174)
  5  model al-feat: -0.154086 (tension=4.98511)
  6  model al-feat: -0.153256 (tension=5.02172)
  7  model al-feat: -0.152805 (tension=5.04172)
  8  model al-feat: -0.152558 (tension=5.05269)
     final tension: 5.05874
ITERATION 3
..
  log_e likelihood: -27011.6
  log_2 likelihood: -38969.6
     cross entropy

- Extract target backtranslated words

In [141]:
import re

# Count unique translated words to the ambiguous words in backtranslations per source sentence
def count_unique_words_alignment_backtranslations(position, sentencesN, sourceIn, backtranslationsIn, alignmentsIn_translation, alignmentsIn_backtranslation, output):
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
       
    lineNumber = 0
    counter = 0
    positions_ambiguous_words_translations = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn_translation, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                positions_ambiguous_words_translations.append([int(index) for index in re.findall(regex, line)])
            else:
                positions_ambiguous_words_translations.append([999])
            lineNumber += 1
    
    # List with backtranslations
    backtranslations = []
    with open(backtranslationsIn, 'r') as fin:
        for line in fin:
            backtranslations.append(line.strip())

    lineNumber = 0
    counter = 0
    indices = []
    with open(alignmentsIn_backtranslation, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            positions = positions_ambiguous_words_translations[counter] # exact positions of ambiguous words
            list_indices = []
            for position in positions:
                regex = r"" + str(position) + r"-(\d)"
                if re.findall(regex, line): 
                    list_indices.extend([int(index) for index in re.findall(regex, line)])
                else:
                    list_indices.extend([999])
            indices.append(list_indices)
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    backtranslations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    backtranslated_ambiguous_words = set() # set forces uniqueness
    for backtranslation in backtranslations:
        tokens = backtranslation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                #print(lineNumber)
                #print(tokens[ind])
                #print(ind)
                backtranslated_ambiguous_words.add(tokens[ind])
        lineNumber += 1
        if (lineNumber % 10 == 0):
                backtranslations_ambiguous_words.append(backtranslated_ambiguous_words)
                backtranslated_ambiguous_words = set()



    #print(backtranslations_ambiguous_words)
    print(len(backtranslations_ambiguous_words))

    # Here we need to merge the sets for every 10 sets, because we want to see unique words in the nbest 100 backtranslation
    backtranslations_ambiguous_words_reduced = []
    backtranslated_ambiguous_words = set() # set forces uniqueness
    counter = 0
    for set_words in backtranslations_ambiguous_words:
        backtranslated_ambiguous_words.update(set_words)
        counter += 1
        if (counter % 10 == 0):
            backtranslations_ambiguous_words_reduced.append(backtranslated_ambiguous_words)
            backtranslated_ambiguous_words = set()

    print(len(backtranslations_ambiguous_words_reduced)) 
    
    # Add results to file

    ambiguous_words = []
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0                
    with open(output, 'w') as fout:
        while count < sentencesN:
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(backtranslations_ambiguous_words_reduced[count]), end='\n', file=fout)
            count += 1

    unique_backtranslations = 0
    for set_words in backtranslations_ambiguous_words_reduced:
        unique_backtranslations += len(set_words)
        
    return unique_backtranslations/sentencesN

In [142]:
print(count_unique_words_alignment_backtranslations(1, 20, 'tok.en_original.en', 'hyp_original_back.txt', 'original_source-target_en-de_fast-aligned.txt', 'original_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_original.txt'))
print(count_unique_words_alignment_backtranslations(2, 20, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt', 'disambiguated_male_source-target_en-de_fast-aligned.txt', 'disambiguated_male_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_disambiguated_male.txt')) # positions is 2 because of gender word
print(count_unique_words_alignment_backtranslations(2, 20, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt', 'disambiguated_female_source-target_en-de_fast-aligned.txt', 'disambiguated_female_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_disambiguated_female.txt'))

200
20
6.15
200
20
5.95
200
20
6.75


## awesome_align

- Extract the position of the translated ambiguous word from each sentence

- Word alignement

In [143]:
# ??? How to set model correctly
# MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
!awesome-align \
    --output_file "original_translation-back_en-de_awesome-aligned.txt" \
    --data_file "original_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "disambiguated_male_translation-back_en-de_awesome-aligned.txt" \
    --data_file "disambiguated_male_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "disambiguated_female_translation-back_en-de_awesome-aligned.txt" \
    --data_file "disambiguated_female_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

print("Finished alignment.")

Loading the dataset...
Extracting: 2000it [00:03, 623.14it/s]
Loading the dataset...
Extracting: 2000it [00:03, 596.01it/s]
Loading the dataset...
Extracting: 2000it [00:03, 630.16it/s]
Finished alignment.


- Extract target backtranslated words

In [144]:
print(count_unique_words_alignment_backtranslations(1, 20, 'tok.en_original.en', 'hyp_original_back.txt', 'original_source-target_en-de_awesome-aligned.txt', 'original_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_original.txt'))
print(count_unique_words_alignment_backtranslations(2, 20, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt', 'disambiguated_male_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_disambiguated_male.txt')) # positions is 2 because of gender word
print(count_unique_words_alignment_backtranslations(2, 20, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt', 'disambiguated_female_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_disambiguated_female.txt'))

200
20
5.55
200
20
5.55
200
20
6.4


# Word alignement (translation-translation)

## Tercom alignement (borrowed from Tu)
- https://github.com/TuAnh23/Perturbation-basedQE/blob/master/align_and_analyse_ambiguous_trans.py#L54-L92

In [ ]:
!git clone https://github.com/TuAnh23/Perturbation-basedQE.git

In [145]:
%cd $TERCOM

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_male/Perturbation-basedQE


In [146]:
import align_and_analyse_ambiguous_trans as tercom
import pandas as pd

def count_unique_words_tercom_alignment(position, sentencesN, sourceIn, backtranslationsIn):
    # List with source sentences; output 100 times to match backtranslation size
    source = []
    with open(PATH + "/" + sourceIn, 'r') as fin:
        for line in fin:
            for i in range(100): # append the source sentence 100 times to match backtranslations later
                source.append(line.strip().split()) # split() tokenizes the sentence, because tercom expects tokens     

    print(len(source))

    # List with original backtranslations
    backtranslations = []
    with open(PATH + "/" + backtranslationsIn, 'r') as fin:
        for line in fin:
            backtranslations.append(line.strip().split())

    print(len(backtranslations))
    
    # Generate alignments
    alignments = tercom.tercom_alignment(source, backtranslations)
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
        
    # Extract target translated words to source words
    lineNumber = 0
    counter = 0
    indices = []
    for align in alignments:
        position = positions_ambiguous_words[counter] # exact position of ambiguous word
        indices.append([item[1] for item in (item for item in align if not(pd.isna(item[0]))) if item[0] == position][0])
        lineNumber += 1
        if (lineNumber % 100 == 0):
            counter += 1

    print(len(indices))

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = set() # set forces uniqueness
    for backtranslation in backtranslations:
        backtranslation_index = backtranslations.index(backtranslation)
        if not(pd.isna(indices[backtranslation_index])):
            translated_ambiguous_words.add(backtranslation[indices[backtranslation_index]])
        lineNumber += 1
        if (lineNumber % 100 == 0):
            translations_ambiguous_words.append(translated_ambiguous_words)
            translated_ambiguous_words = set()

    #print(translations_ambiguous_words)
    #print(len(translations_ambiguous_words))

    unique_translations = 0
    for set_words in translations_ambiguous_words:
        unique_translations += len(set_words)
        
    return unique_translations/sentencesN

In [147]:
print(count_unique_words_tercom_alignment(1, 20, 'tok.en_original.en', 'hyp_original_back.txt'))
print(count_unique_words_tercom_alignment(2, 20, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt'))
print(count_unique_words_tercom_alignment(2, 20, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt'))

2000
2000
2000
7.4
2000
2000
2000
8.75
2000
2000
2000
11.1


In [148]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_words/Professions/Female


# Word occurrence

## Translation

In [149]:
def extract_word_translations(filename_tokenized, filename_translations, filename_out, filename_alignments):
    """
    Match alignment indices from translation to backtranslation
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_alignments, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    translations = []
    with open(filename_translations, 'r') as fin:
         for line in fin:
                translations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 20
    counter = 0
    for i in range(0, 20): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 9):
                alignments = indices_translation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(translations[counter + f])):
                                 words_set.add(translations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 10

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 20:
            print(source[count] + ' | ' + str(target_words[count]), end='\n', file=fout)
            count += 1

In [150]:
def count_word_translations(filename_tokenized, filename_translations, filename_out, filename_alignments):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_alignments, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    translations = []
    with open(filename_translations, 'r') as fin:
         for line in fin:
                translations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 20
    counter = 0
    for i in range(0, 20): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 9):
                alignments = indices_translation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(translations[counter + f])):
                                 words_set.add(translations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 10

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 20:
            print(source[count] + ' | ' + str([len(target_set) for target_set in target_words[count]]), end='\n', file=fout)
            count += 1

In [151]:
extract_word_translations('tok.en_original.en', 'hyp_original.txt', 'translations_words_original.txt', 'original_source-target_en-de_awesome-aligned.txt')
count_word_translations('tok.en_original.en', 'hyp_original.txt', 'translations_words_original_occurrence.txt', 'original_source-target_en-de_awesome-aligned.txt')

extract_word_translations('tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'translations_words_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt')
count_word_translations('tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'translations_words_disambiguated_male_occurrence.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt')

extract_word_translations('tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'translations_words_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt')
count_word_translations('tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'translations_words_disambiguated_female_occurrence.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt')

## Backtranslation

In [152]:
def extract_alignment_indices_backtranslation(filename_translations, filename_backtranslations):
    """
    Extract alignment indices
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_translations, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
       
    # Match alignement indices from translation to backtranslation
    lineNumber = 0
    counter = 0
    indices_backtranslation = []
    with open(filename_backtranslations, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            alignement_tokens = line.split()
            indices_line = []
            for index_list in indices_translation[counter]:
                index_matches = []
                for index in index_list:
                    regex = r"" + str(index) + r"-(\d)"
                    if re.findall(regex, line): 
                        index_matches.extend([int(i) for i in re.findall(regex, line)])
                    else:
                        index_matches.extend([999])
                indices_line.append(index_matches)
            indices_backtranslation.append(indices_line)
            lineNumber += 1 
    return indices_backtranslation

In [153]:
def extract_word_backtranslations(filename_tokenized, filename_backtranslations, filename_out, indices_backtranslation):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    backtranslations = []
    with open(filename_backtranslations, 'r') as fin:
         for line in fin:
                backtranslations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 20
    counter = 0
    for i in range(0, 20): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 99):
                alignments = indices_backtranslation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(backtranslations[counter + f])):
                                 words_set.add(backtranslations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 100

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 20:
            print(source[count] + ' | ' + str(target_words[count]), end='\n', file=fout)
            count += 1

In [154]:
def count_word_backtranslations(filename_tokenized, filename_backtranslations, filename_out, indices_backtranslation):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    backtranslations = []
    with open(filename_backtranslations, 'r') as fin:
         for line in fin:
                backtranslations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 20
    counter = 0
    for i in range(0, 20): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 99):
                alignments = indices_backtranslation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(backtranslations[counter + f])):
                                 words_set.add(backtranslations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 100

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 20:
            print(source[count] + ' | ' + str([len(target_set) for target_set in target_words[count]]), end='\n', file=fout)
            count += 1

In [155]:
indices_original = extract_alignment_indices_backtranslation('original_source-target_en-de_awesome-aligned.txt', 'original_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_original.en', 'hyp_original_back.txt', 'backtranslations_words_original.txt', indices_original)
count_word_backtranslations('tok.en_original.en', 'hyp_original_back.txt', 'backtranslations_words_original_occurrence.txt', indices_original)

indices_disambiguated_male = extract_alignment_indices_backtranslation('disambiguated_male_source-target_en-de_awesome-aligned.txt', 'disambiguated_male_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt', 'backtranslations_words_disambiguated_male.txt', indices_disambiguated_male)
count_word_backtranslations('tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt', 'backtranslations_words_disambiguated_male_occurrence.txt', indices_disambiguated_male)

indices_disambiguated_female = extract_alignment_indices_backtranslation('disambiguated_female_source-target_en-de_awesome-aligned.txt', 'disambiguated_female_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt', 'backtranslations_words_disambiguated_female.txt', indices_disambiguated_female)
count_word_backtranslations('tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt', 'backtranslations_words_disambiguated_female_occurrence.txt', indices_disambiguated_female)

# Gender statistics

In [156]:
from enum import Enum

class GENDER(Enum):
    """
    Enumerate possible genders.
    Ignore option resolves to words that should be ignored in particular language
    """
    male = 0
    female = 1
    neutral = 2
    unknown = 3
    ignore = 4
    
# ??? These are not always correct; 'der' could be Dativ or Genitiv for female, 'die' could be plural
# !!! There isn't always an article
DE_DETERMINERS = {"der": GENDER.male, "ein": GENDER.male, "dem": GENDER.male, "den": GENDER.male, 
                  "einen": GENDER.male, "des": GENDER.male, "er": GENDER.male, "seiner": GENDER.male,
                  "ihn": GENDER.male, "seinen": GENDER.male, "ihm": GENDER.male, "ihren": GENDER.male,
                  "die": GENDER.female, "eine": GENDER.female, "einer": GENDER.female, "seinem": GENDER.male,
                  "ihrem": GENDER.male, "sein": GENDER.male,
                  "sie": GENDER.female, "seine": GENDER.female, "ihrer": GENDER.female, 
                  "ihr": GENDER.neutral, "ihre": GENDER.neutral, "das": GENDER.neutral,
                  "jemanden": GENDER.neutral}

def get_german_determiners(words):
    """
    Get a list of (gender)
    given a list of words.
    """
    determiners = []
    for (word_ind, word) in enumerate(words):
        word = word.lower()
        if word in DE_DETERMINERS:
            determiners.append((DE_DETERMINERS[word].name))
    return determiners

In [157]:
dets = get_german_determiners(["dem"])
print(dets)

['male']


- Calculate gender based on the articles of unique words: how many of the sentences produce both genders, female and male

In [158]:
import re

# Extract articles of target tranlsated words
def extract_articles(position, sentencesN, translationsIn, alignmentsIn, sourceIn, output):
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
    
    # List with original translations
    translations_original = []
    with open(translationsIn, 'r') as fin:
        for line in fin:
            translations_original.append(line.strip())


    lineNumber = 0
    counter = 0
    indices = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                indices.append([int(index) for index in re.findall(regex, line)])
            else:
                indices.append([999])
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = set() # set forces uniqueness
    for translation in translations_original:
        tokens = translation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                translated_ambiguous_words.add(tokens[0]) # extract articles; currently assume index 0 for article position, TODO
        lineNumber += 1
        if (lineNumber % 10 == 0):
                translations_ambiguous_words.append(translated_ambiguous_words)
                translated_ambiguous_words = set()


    #print(len(translations_ambiguous_words))
    
    # Add results to file

    # List with original source sentences
    source = []
    ambiguous_words = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0  
    genders = []
    male = []
    female = []
    with open(output, 'w') as fout:
        while count < sentencesN:
            #print(translations_ambiguous_words[count])
            genders.append(set(get_german_determiners(translations_ambiguous_words[count])))
            male.append("male" in get_german_determiners(translations_ambiguous_words[count]))
            female.append("female" in get_german_determiners(translations_ambiguous_words[count]))
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(get_german_determiners(translations_ambiguous_words[count])), end='\n', file=fout)
            count += 1
            
    return (sum(1 for i in genders if ('male' in i and 'female' in i)), 
            male.count(True), female.count(True))

In [159]:
print(extract_articles(1, 20, 'hyp_original.txt', 'original_source-target_en-de_awesome-aligned.txt', 'tok.en_original.en', 'unique-words_translations_original_articles.txt'))
print(extract_articles(2, 20, 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt', 'tok.en_disambiguated_male.en', 'unique-words_translations_disambiguated_male_articles.txt'))
print(extract_articles(2, 20, 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt', 'tok.en_disambiguated_female.en', 'unique-words_translations_disambiguated_female_articles.txt'))

(14, 19, 15)
(10, 20, 10)
(0, 0, 20)


- Calculate gender in percentage for each sentence: percent of "male" vs. female in translations for each sentence

In [160]:
import re

# Extract articles of target tranlsated words
def extract_articles_percent(position, sentencesN, translationsIn, alignmentsIn, sourceIn, output):
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
    
    # List with original translations
    translations_original = []
    with open(translationsIn, 'r') as fin:
        for line in fin:
            translations_original.append(line.strip())


    lineNumber = 0
    counter = 0
    indices = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                indices.append([int(index) for index in re.findall(regex, line)])
            else:
                indices.append([999])
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = [] 
    for translation in translations_original:
        tokens = translation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                translated_ambiguous_words.append(tokens[0]) # extract articles; currently assume index 0 for article position, TODO
        lineNumber += 1
        if (lineNumber % 10 == 0):
                translations_ambiguous_words.append(translated_ambiguous_words)
                translated_ambiguous_words = []


    #print(len(translations_ambiguous_words))
    
    # Add results to file

    # List with original source sentences
    source = []
    ambiguous_words = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0  
    genders = []
    male = []
    female = []
    with open(output, 'w') as fout:
        while count < sentencesN:
            #print(translations_ambiguous_words[count])
            genders.append(get_german_determiners(translations_ambiguous_words[count]))
            male.append("male" in get_german_determiners(translations_ambiguous_words[count]))
            female.append("female" in get_german_determiners(translations_ambiguous_words[count]))
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(get_german_determiners(translations_ambiguous_words[count])), end='\n', file=fout)
            count += 1
     
    #print(genders)
    return (sum([i.count('male')/10 for i in genders])/sentencesN*100, 
            sum([i.count('female')/10 for i in genders])/sentencesN*100)

In [161]:
print(extract_articles_percent(1, 20, 'hyp_original.txt', 'original_source-target_en-de_awesome-aligned.txt', 'tok.en_original.en', 'unique-words_translations_original_articles.txt'))
print(extract_articles_percent(2, 20, 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt', 'tok.en_disambiguated_male.en', 'unique-words_translations_disambiguated_male_articles.txt'))
print(extract_articles_percent(2, 20, 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt', 'tok.en_disambiguated_female.en', 'unique-words_translations_disambiguated_female_articles.txt'))

(65.99999999999999, 34.0)
(79.00000000000001, 21.000000000000004)
(0.0, 100.0)


# Stemming

In [642]:
import spacy

nlp = spacy.load('de')
doc = nlp('Entwicklerin')
for token in doc:
    print(token, token.lemma, token.lemma_)
    
doc = nlp('Entwickler')
for token in doc:
    print(token, token.lemma, token.lemma_)

OSError: [E049] Can't find spaCy data directory: 'None'. Check your installation and permissions, or use spacy.util.set_data_path to customise the location if necessary.

In [631]:
from nltk.stem.cistem import Cistem
 
stemmer = Cistem(case_insensitive=True)
s1 = "Entwicklerin"
print(stemmer.stem(s1))
s2 = "Entwickler"
print(stemmer.stem(s2))

entwickleri
entwickl


In [586]:
!pip install german-lemmatizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [623]:
from german_lemmatizer import lemmatize

lemmatize(
    ['Johannes war ein guter Schüler', 'Sabiene sang zahlreiche Lieder'],
    working_dir='*',
    chunk_size=10000,
    n_jobs=1,
    escape=False,
    remove_stop=False)

<generator object lemmatize at 0x7f229d89be40>

In [596]:
!pip install HanTa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 20.5 MB/s eta 0:00:0000:0100:01


In [632]:
from HanTa import HanoverTagger as ht

tagger = ht.HanoverTagger('morphmodel_ger.pgz')

print(tagger.analyze('Ärztin'))
print(tagger.analyze('Arzt'))

('Ärztin', 'NN')
('Arzt', 'NN')


In [609]:
!pip install -U textblob-de

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 8.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 23.0 MB/s eta 0:00:00


In [610]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /home/vzhekova/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/vzhekova/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/vzhekova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vzhekova/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     /home/vzhekova/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/vzhekova/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [621]:
from textblob_de import TextBlobDE

word = 'Ärztin'
w = TextBlobDE(word)
print(w.words.lemmatize())

word = 'Arzt'
w = TextBlobDE(word)
print(w.words.lemmatize())

['Ärztin']
['Arzt']


In [ ]:
!pip install spacy-udpipe

In [638]:
import spacy_udpipe

spacy_udpipe.download("de") # download English model

text = "Entwicklerin"
nlp = spacy_udpipe.load("de")

doc = nlp(text)
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_)

ImportError: cannot import name 'Language' from 'spacy' (/home/vzhekova/miniconda3/envs/nmt/lib/python3.8/site-packages/spacy/__init__.py)